# Preprocess Weather Data 

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium

In [4]:
# Read NYC weather data 
wdata_2023 = pd.read_csv('../data/landing/external/NYC_weather_2023.csv')
wdata_2024 = pd.read_csv('../data/landing/external/NYC_weather_2024.csv')

/var/folders/gf/ml3nsztn0cb376sq34z0ll1r0000gn/T/ipykernel_32240/2561988319.py:2: DtypeWarning: Columns (36,42,43,48,52,88) have mixed types. Specify dtype option on import or set low_memory=False.
  wdata_2023 = pd.read_csv('../data/landing/external/NYC_weather_2023.csv')


In [9]:
wdata_2023.head(10)

,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,...,OD1,OE1,OE2,OE3,RH1,RH2,RH3,WA1,REM,EQD
0,72505394728,2023-01-01T00:06:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-16,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET11812/31/22 19:06:03 SPECI KNYC 010006Z 000...,NaN
1,72505394728,2023-01-01T00:18:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-16,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10612/31/22 19:18:03 SPECI KNYC 010018Z 030...,NaN
2,72505394728,2023-01-01T00:31:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-16,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET11112/31/22 19:31:03 SPECI KNYC 010031Z 000...,NaN
3,72505394728,2023-01-01T00:44:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-16,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10212/31/22 19:44:03 SPECI KNYC 010044Z 000...,NaN
4,72505394728,2023-01-01T00:49:00,6,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-16,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET09712/31/22 19:49:03 SPECI KNYC 010049Z 000...,NaN
5,72505394728,2023-01-01T00:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET12112/31/22 19:51:03 METAR KNYC 010051Z 000...,NaN
6,72505394728,2023-01-01T01:19:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-16,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET11812/31/22 20:19:03 SPECI KNYC 010119Z 000...,NaN
7,72505394728,2023-01-01T01:27:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-16,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET11412/31/22 20:27:03 SPECI KNYC 010127Z VRB...,NaN
8,72505394728,2023-01-01T01:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET11412/31/22 20:51:03 METAR KNYC 010151Z 250...,NaN
9,72505394728,2023-01-01T02:49:00,6,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-16,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET09612/31/22 21:49:03 SPECI KNYC 010249Z VRB...,NaN


In [10]:
wdata_2024.head(10) 

,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,...,OD1,OE1,OE2,OE3,RH1,RH2,RH3,WA1,REM,EQD
0,72505394728,2024-01-01T00:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10012/31/23 19:51:03 METAR KNYC 010051Z 250...,NaN
1,72505394728,2024-01-01T01:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET09612/31/23 20:51:03 METAR KNYC 010151Z 000...,NaN
2,72505394728,2024-01-01T02:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10212/31/23 21:51:03 METAR KNYC 010251Z VRB...,NaN
3,72505394728,2024-01-01T03:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET09612/31/23 22:51:03 METAR KNYC 010351Z 250...,NaN
4,72505394728,2024-01-01T04:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET11712/31/23 23:51:03 METAR KNYC 010451Z 260...,NaN
5,72505394728,2024-01-01T04:59:00,6,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",SOD,KNYC,V030,...,NaN,"1,24,00939,300,9999,5","2,24,00581,300,9999,5","3,24,00237,999,9999,5","024,X,060,D,4","024,N,047,D,4","024,M,050,D,4",NaN,SOD77024 HR PRECIPITATION (IN): 0.00 SNOW DEPT...,NaN
6,72505394728,2024-01-01T04:59:00,6,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",SOM,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SOM796PCP MTOT:6.71 PCP GT 24HRS:2.46 DATE(S):...,R01 6.717ABP070R02 2.467ADP081R03 44.67KBM0...
7,72505394728,2024-01-01T05:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET12101/01/24 00:51:02 METAR KNYC 010551Z 260...,NaN
8,72505394728,2024-01-01T06:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET09601/01/24 01:51:02 METAR KNYC 010651Z VRB...,NaN
9,72505394728,2024-01-01T07:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET09601/01/24 02:51:02 METAR KNYC 010751Z 000...,NaN


In [9]:
# Convert the 'Date' column to datetime format
wdata['Date'] = pd.to_datetime(wdata['Date'])

# Filter data between June 2023 and May 2024
wdata = wdata[(wdata['Date'] >= '2023-06-01') & (wdata['Date'] <= '2024-05-31')]

KeyError: 'Date'